In [1]:
import tqdm 
import pandas as pd
import akshare as ak

def filter_shsz(v):
    code,name = v["code"],v["name"]
    if code.startswith("68") or code.startswith("8")or code.startswith("4"):
        return False
    elif "ST" in name or "退市" in name or "C" in name:
        return False
    elif code[0] not in ("0","3","6"):
        return False
    return True

def reform_shsz(code):
    if code.startswith("3") or code.startswith("00"):
        return "sz"+code
    elif code.startswith("6"):
        return "sh"+code
    return code
    
    
    
df = ak.stock_zh_a_spot_em()
df = df.rename(columns={"代码":"code","名称":"name"})
df = df[df.apply(filter_shsz,axis=1)]
df["code"] = df["code"].apply(reform_shsz)
cdf = df[["code","name"]]
lst = []
hdf = pd.read_csv("stock_hist.csv")
hs = set(hdf["code"].drop_duplicates())
cdf = cdf[~cdf.code.isin(hs)]
lst = [hdf]
bar = tqdm.tqdm(total =cdf.shape[0])

for i,(idx,v)in enumerate(cdf.iterrows()):
    bar.update()
    if i>1000:
        break
    try:
        tdf = ak.stock_zh_a_cdr_daily(symbol=v["code"], start_date="20241116")[["date","open","high","low","close","volume","amount"]]
        tdf[["code","name"]]=(v["code"],v["name"])
        lst.append(tdf)
    except:
        print(v)
if len(lst)>1:
    hdf = pd.concat(lst)
    hdf['code'] = hdf['code'].astype(str)
    hdf.to_csv("stock_hist.csv",index=False)


/Users/john/anaconda3/lib/python3.11/site-packages/akshare/__init__.py:2977: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.2.0 及以上版本！
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'stock_hist.csv'

In [1]:
import akshare as ak
import datetime

def filter_shsz(v):
    code,name = v["code"],v["name"]
    if code.startswith("68") or code.startswith("8")or code.startswith("4"):
        return False
    elif "ST" in name or "退" in name or "C" in name:
        return False
    elif code[0] not in ("0","3","6"):
        return False
    return True


def reform_shsz(code):
    if code.startswith("3") or code.startswith("00"):
        return "sz"+code
    elif code.startswith("6"):
        return "sh"+code
    return code
    
today, bias = datetime.datetime.now(), 0
cdate = today+datetime.timedelta(days=-bias)
cdate = cdate.strftime('%Y-%m-%d')
cdf = ak.stock_zh_a_spot_em()
cdf = cdf.rename(columns={"代码":"code","名称":"name"})
cdf = cdf[cdf.apply(filter_shsz,axis=1)]
cdf["code"] = cdf["code"].apply(reform_shsz)
cdf = cdf.rename(columns={"今开":"open","最低":"low","最高":"high","最新价":"close",
                         "成交量":"volume","成交额":"amount"})
cdf["date"]=cdate
frac = 1.5
cdf["volume"] = cdf["volume"]*100*frac
cdf["amount"] = cdf["amount"]*frac
cdf = cdf[["date","code","name","open","low","high","close","volume","amount"]]
# hdf = pd.read_csv("stock_hist.csv")
# df = pd.concat([hdf,cdf])
# cdf.to_csv("stock_today.csv",index=False)
cdf.to_csv("stock_today.csv",index=False)
cdf.head()

/Users/john/anaconda3/lib/python3.11/site-packages/akshare/__init__.py:2977: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.2.0 及以上版本！
  warnings.warn(


,date,code,name,open,low,high,close,volume,amount
1,2025-01-06,sz300434,金石亚药,8.30,8.30,9.46,9.46,39759450.0,3.611538e+08
3,2025-01-06,sz301157,华塑科技,41.91,41.65,48.78,48.78,9382050.0,4.486805e+08
5,2025-01-06,sz300766,每日互动,14.51,14.46,17.65,17.14,155909550.0,2.663014e+09
6,2025-01-06,sz300158,振东制药,4.15,4.15,4.93,4.84,96580200.0,4.453419e+08
8,2025-01-06,sz300147,香雪制药,9.90,9.90,11.50,11.29,105789000.0,1.138466e+09


In [13]:
import pandas as pd

def save_dt():
    def get_weekday(cdate):
        holidays = set(["2025-01-01"])
        for i in range(1,5):
            ttm = today+datetime.timedelta(days=-bias-i)
            if ttm.weekday() <5 and ttm.strftime("%Y-%m-%d") not in holidays:
                return ttm

    today = datetime.datetime.now()
    cdate = today+datetime.timedelta(days=-bias)
    pdate = get_weekday(cdate)
    cdate,pdate = cdate.strftime('%Y-%m-%d'),pdate.strftime('%Y-%m-%d')
    df = pd.read_csv(f"daily/stock_{pdate}.csv")
    cdf = pd.read_csv("stock_today.csv")
    df = pd.concat([df,cdf])
    df.to_csv(f"daily/stock_{cdate}.csv",index=False)
    cdf.to_csv(f"daily/stock_daily_{cdate}.csv",index=False)

def clean_data(pattern="stock_\d{4}-\d{2}-\d{2}.csv"):
    import os, re
    lst = []
    for r,d, files in os.walk("./daily"):
        for f in files:
            if re.match(pattern,f):
                lst.append(os.path.join(r,f))
    for f in sorted(lst)[:-2]:
        os.remove(f)

save_dt()
clean_data()